In [26]:
! pip install -U hopsworks --quiet

In [27]:
import pandas as pd
import hopsworks
import joblib

project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3342
Connected. Call `.close()` to terminate connection gracefully.


## Load the Model from Model Registry

In [66]:
mr = project.get_model_registry()
model = mr.get_model("car_prices", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/car_prices_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.

In [117]:
try:
  feature_view = fs.get_feature_view(name="car_prices_inf", version=1)
except:
  car_fg  = fs.get_feature_group(name="car_prices_inference", version=1)
  query = car_fg .select_all()
  feature_view = fs.create_feature_view(name="car_prices_inf",
                                        version=1,
                                        description="Read from car price dataset for inference",
                                        labels=["selling_price"],
                                        query=query)

In [118]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)

In [119]:
import datetime
from PIL import Image

batch_data = feature_view.get_batch_data()

td_car = batch_data.sample()

td_car_pred = td_car.drop('full_name', axis = 1)

predictions = model.predict(td_car_pred)

In [120]:
td_car

,full_name,km_driven,mileage,engine,max_power,seats,age,seller_type_dealer,seller_type_individual,seller_type_trustmark_dealer,fuel_type_cng,fuel_type_diesel,fuel_type_electric,fuel_type_petrol,transmission_type_automatic,transmission_type_manual
91,Maruti Swift ZDI BSIV,123000,25.2,1248.0,74.0,5.0,7,1,0,0,0,1,0,0,0,1


In [121]:
import pandas as pd

monitor_fg = fs.get_or_create_feature_group(name="car_predictions",
                                  version=1,
                                  primary_key=["datetime"],
                                  description="Car Price Prediction/Outcome Monitoring"
                                 )

In [122]:
from datetime import datetime
now = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")

data = {
    'prediction': predictions[-1],
    'label': td_car.iloc[0][0],
    'datetime': [now],
}
monitor_df = pd.DataFrame(data)
monitor_fg.insert(monitor_df)

Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3342/jobs/named/car_predictions_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fdbcfb52910>, None)

In [123]:
history_df = monitor_fg.read()
history_df

,prediction,label,datetime
0,19.2094,Mercedes-Benz GLA Class 200 d Sport,"11/18/2022, 04:02:57"
1,6.3665,Hyundai Grand i10 Nios AMT Sportz,"11/18/2022, 04:46:20"
2,8.1128,Mahindra TUV 300 mHAWK100 T8,"11/18/2022, 05:14:08"
3,5.9010,Maruti Swift ZDI BSIV,"11/18/2022, 05:17:02"


In [36]:
! pip install dataframe-image --quiet

In [124]:
import dataframe_image as dfi

df_recent = history_df.tail(5)
 
dfi.export(df_recent, 'assets/df_recent.png', table_conversion = 'matplotlib')

In [116]:
# import datetime
# start_date = (datetime.datetime.now() - datetime.timedelta(hours=240)) 
# end_date = (datetime.datetime.now()) 
# print(start_date)
# print(end_date)